In [1]:
import pandas as pd
from pyecharts import Boxplot
from pyecharts import Bar 
from pyecharts import Pie
from pyecharts import Scatter
from pyecharts import WordCloud
import numpy as np
from snownlp import SnowNLP
import jieba
# from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from matplotlib import pyplot as plt
import os
import collections
import seaborn as sns

In [2]:
title_class = '厨房用品、厨房电器'

In [3]:
# 载入数据
data = pd.read_csv('./data_list/data_list_after_groupby.csv', sep=',', header=0, encoding='utf-8',engine='python',dtype=str)
# data = data.iloc[:10001, :]
data = data[data['title'] == title_class]
print('导出数据共有{}行，{}列！'.format(data.shape[0], data.shape[1]))
print('读取数据源成功！！')   
    
# 按小类箱线图检测异常值  ——  pyecharts
title_name = data['cate'].unique()
title_name = list(title_name)
title_price = []
for title in title_name:
    pro_price = data[data['cate']==title]['pro_price'].astype(float).unique()
    pro_price = list(pro_price)
    title_price.append(pro_price)
# print(f'title_price是 {np.array(title_price).shape[0]}维数据')
file_name = 'boxplot_' + title_class + '_output.html'
boxplot = Boxplot('按小类箱线图检测异常值',width=1500, height=400)
x_axis = title_name
y_axis = title_price
_yaxis = boxplot.prepare_data(y_axis)
boxplot.add('小类', x_axis, _yaxis,xaxis_rotate=45,width=1600,height=800)
boxplot.render(os.path.join('./data_list',file_name))
boxplot

导出数据共有24203行，8列！
读取数据源成功！！


In [4]:
# 单变量：评分区间直方图  ——  pyecharts
sentiment_score = data['sentiment_score'].astype(float)
group = [0, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]
data['group_num'] = pd.cut(sentiment_score, group, labels=['0', '0.1', '0.2', '0.3','0.4','0.5','0.6','0.7','0.8','0.9','1.0'],right=False)
data_hist = pd.pivot_table(data,index=['group_num'],values=['sentiment_score'],aggfunc=[len])
data_hist.reset_index(inplace=True)
data_hist.sort_values(by='group_num',ascending=True,inplace=True)
x_axis_list = list(data_hist['group_num'])
y_axis_list = list(data_hist[('len','sentiment_score')])
# print(x_axis_list,y_axis_list)
bar = Bar('评分区间直方图 ')
x_axis = x_axis_list
y_axis = y_axis_list
bar.add('评分区间', x_axis, y_axis,bar_category_gap=0,is_label_show=True)
file_name = 'hist_' + title_class + '_output.html'
bar.render(os.path.join('./data_list',file_name))
bar

In [5]:
# 单变量：评论分类饼图  ——  pyecharts
pie_data = pd.pivot_table(data,index=['group'],values=['item_id'],aggfunc=[len])
pie_data.reset_index(inplace=True)
pie_data.sort_values(by=('len','item_id'),ascending=False,inplace=True)
size = pie_data[('len','item_id')]
labels = pie_data['group']
print(labels,size)
pie = Pie('评论分类饼图', title_pos='center')
attr = list(labels)
v1 = list(size)
pie.add('', attr, v1,radius=[40, 75],label_text_color=None,is_label_show=True,legend_orient='vertical',legend_pos='left')
file_name = 'pie_' + title_class + '_output.html'
pie.render(os.path.join('./data_list',file_name))
pie

1    好评
2    差评
0    中评
Name: group, dtype: object 1    15718
2     5503
0     2982
Name: (len, item_id), dtype: int64


In [6]:
# 单变量：小类条形图  ——  pyecharts
hbar_data = pd.pivot_table(data,index=['cate'],values=['item_id'],aggfunc=[len])
hbar_data.reset_index(inplace=True)
hbar_data.sort_values(by=('len','item_id'),ascending=True,inplace=True)
title_name = hbar_data['cate']
size = hbar_data[('len','item_id')]
x_axis_list = list(title_name)
y_axis_list = list(size)
bar = Bar('小类条形图 ',width=800, height=1000)
x_axis = x_axis_list
y_axis = y_axis_list
bar.add('评论数', x_axis, y_axis,is_convert=True,is_label_show=True,label_pos='right')
file_name = 'hbar_' + title_class + '_output.html'
bar.render(os.path.join('./data_list',file_name))
bar

In [7]:
# 双变量：小类、评论分数柱状图  ——  pyecharts
data['sentiment_score'] = data['sentiment_score'].astype(float)
bar_data = pd.pivot_table(data,index=['cate'],values=['sentiment_score'],aggfunc=[np.mean])
bar_data.reset_index(inplace=True)
bar_data.sort_values(by=('mean','sentiment_score'),ascending=False,inplace=True)
f = lambda x: '%.2f'%x
bar_data[('mean','sentiment_score')] = bar_data[('mean','sentiment_score')].map(f)
title_name = bar_data['cate']
score = bar_data[('mean','sentiment_score')]
x_axis_list = list(title_name)
y_axis_list = list(score)
bar = Bar('小类评论分数柱状图 ',width=1500, height=400)
x_axis = x_axis_list
y_axis = y_axis_list
bar.add('评论分数', x_axis, y_axis,xaxis_rotate=45,is_label_show=True)
file_name = 'bar_' + title_class + '_output.html'
bar.render(os.path.join('./data_list',file_name))
bar

In [8]:
# 双变量：价格、评论分数散点图  ——  pyecharts
# sentiment_score = data['sentiment_score'].astype(float)
# pro_price = data['pro_price'].astype(float)
# x_axis_list = list(sentiment_score)
# y_axis_list = list(pro_price)
# x_axis = x_axis_list
# y_axis = y_axis_list
# scatter = Scatter('价格评论分数散点图')
# scatter.add('', x_axis, y_axis, symbol_size=5,is_visualmap=True)
# scatter.render('./data_list/scatter_all_output.html')
# scatter

In [9]:
# 词云  ——  pyecharts 
data_cut_words = pd.DataFrame(data.astype(str))
# data_cut_words['cut_comment'] = data_duplicate_words['seg_comment'].apply(cut_words)
# stopwords = load_stopwords()
stopwords = [line.strip() for line in open('./常见中文停用词表.txt', encoding='gbk').readlines()]
# 保存全局分词，用于词频统计
segments = []
# 普通分词及停用词判断
for index, row in data_cut_words.iterrows():
    content = row[5]
    # TextRank 关键词抽取，只获取固定词性
    words = jieba.cut(content,cut_all=False)
    # splitedStr = ''
    for word in words:
        # 停用词判断，如果当前的关键词不在停用词库中才进行记录
        if word not in stopwords:
            # 记录全局分词
            segments.append(word)
            # splitedStr += word + ' '
word_counts = collections.Counter(segments) # 对分词做词频统计
word_counts_top10 = word_counts.most_common(50) # 获取前50最高频的词
# print ('文本词频最高的前50个词是: {}'.format(word_counts_top10)) # 输出检查
# 设置中文字体
# font_path = os.path.join('C:\Windows\Fonts', 'FZYTK.TTF')
# 生成词云
# wc = WordCloud(font_path=font_path, margin=2, scale=10, max_words=200, min_font_size=4,random_state=42, background_color='white', max_font_size=150)
# wc.generate_from_frequencies(word_counts)
# 获取文本词排序，可调整 stopwords
# process_word = WordCloud.process_text(wc, text)
# sort = sorted(process_word.items(), key=lambda e: e[1], reverse=True)
# 获取文本词频最高的前50个词
# print('文本词频最高的前50个词是: {}'.format(sort[:50]))

words = []
counts = []

for item in word_counts_top10:
    word = item[0]
    count = item[1]
    words.append(word)
    counts.append(count)
    
attr = words
value = counts
wordcloud = WordCloud(width=800, height=400)
wordcloud.add('评论词云图', attr, value,word_size_range=[10, 100],shape='diamond')
file_name = 'wordcloud_' + title_class + '_output.html'
wordcloud.render(os.path.join('./data_list',file_name))
wordcloud

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.997 seconds.
DEBUG:jieba:Loading model cost 0.997 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
